Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (150000, 28, 28) (150000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [6]:
image_size = 28
num_labels = 10
num_steps = 10001
batch_size = 128
beta = 0.001
offset_list = [np.random.randint(0,100000) for _ in range(25)]

In [7]:


def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (150000, 784) (150000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [3]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Logistic regression with L2 Regularization

In [26]:


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (beta/2)*tf.nn.l2_loss(weights)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Neural Network with L2 Regularization

In [22]:

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Weights (input -> hid, hid -> output)
  weights_01 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid_size]))
  biases_01 = tf.Variable(tf.zeros([hid_size]))
  
  weights_12 = tf.Variable(tf.truncated_normal([hid_size, num_labels]))
  biases_12 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  hid_out = tf.nn.relu(tf.matmul(tf_train_dataset,weights_01) + biases_01)
  logits = tf.matmul(hid_out, weights_12) + biases_12
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (beta/2.)*tf.nn.l2_loss(weights_01) + (beta/2.)*tf.nn.l2_loss(weights_12)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hid_out = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_hid_out, weights_12) +  biases_12)
  
  test_hid_out = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
  test_prediction = tf.nn.softmax(
            tf.matmul(test_hid_out,weights_12) + biases_12)
        

In [23]:

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#10001 Steps
#test accuracy: 89.1% (Logistic Regression)
#test accuracy: 94.3% (Neural Net 1 hidden layer)

Initialized
Minibatch loss at step 0: 492.314240
Minibatch accuracy: 7.8%
Validation accuracy: 33.3%
Minibatch loss at step 500: 124.349945
Minibatch accuracy: 85.9%
Validation accuracy: 80.9%
Minibatch loss at step 1000: 96.463539
Minibatch accuracy: 84.4%
Validation accuracy: 81.4%
Minibatch loss at step 1500: 72.473221
Minibatch accuracy: 84.4%
Validation accuracy: 79.6%
Minibatch loss at step 2000: 57.286072
Minibatch accuracy: 87.5%
Validation accuracy: 83.0%
Minibatch loss at step 2500: 44.651279
Minibatch accuracy: 83.6%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 34.368870
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Minibatch loss at step 3500: 26.489210
Minibatch accuracy: 88.3%
Validation accuracy: 85.3%
Minibatch loss at step 4000: 20.758314
Minibatch accuracy: 87.5%
Validation accuracy: 85.9%
Minibatch loss at step 4500: 16.219786
Minibatch accuracy: 88.3%
Validation accuracy: 86.1%
Minibatch loss at step 5000: 12.672142
Minibatch accuracy: 90.6%
Valida

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [27]:

print("Offsets: ",offset_list)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = offset_list[step%len(offset_list)]
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#Steps 10001
#test accuracy 83.9% (Logistic Regression)(Overfitting)
#test accuracy 89.5% (Neural Network) (Overfitting)

Offsets:  [15958, 14751, 26542, 13569, 23686, 79248, 56272, 90653, 11217, 68634, 43230, 75549, 44776, 62585, 23691, 6094, 27525, 25837, 39670, 48951, 51865, 39897, 83480, 32883, 97918]
Initialized
Minibatch loss at step 0: 18.960087
Minibatch accuracy: 18.8%
Validation accuracy: 19.2%
Minibatch loss at step 500: 2.402911
Minibatch accuracy: 78.9%
Validation accuracy: 72.4%
Minibatch loss at step 1000: 1.592975
Minibatch accuracy: 85.2%
Validation accuracy: 73.0%
Minibatch loss at step 1500: 1.182725
Minibatch accuracy: 88.3%
Validation accuracy: 73.4%
Minibatch loss at step 2000: 0.932421
Minibatch accuracy: 89.8%
Validation accuracy: 74.0%
Minibatch loss at step 2500: 0.768805
Minibatch accuracy: 92.2%
Validation accuracy: 74.4%
Minibatch loss at step 3000: 0.658303
Minibatch accuracy: 92.2%
Validation accuracy: 75.0%
Minibatch loss at step 3500: 0.582152
Minibatch accuracy: 93.0%
Validation accuracy: 75.2%
Minibatch loss at step 4000: 0.528504
Minibatch accuracy: 93.0%
Validation acc

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [28]:
batch_size = 128
hid_size = 1024
beta = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Weights (input -> hid, hid -> output)
  weights_01 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hid_size]))
  biases_01 = tf.Variable(tf.zeros([hid_size]))
  
  weights_12 = tf.Variable(tf.truncated_normal([hid_size, num_labels]))
  biases_12 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  hid_out = tf.nn.dropout(tf.nn.relu(tf.matmul(tf_train_dataset,weights_01) + biases_01),0.5,seed=tf.set_random_seed(123524))
  logits = tf.matmul(hid_out, weights_12) + biases_12
    
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (beta/2.)*tf.nn.l2_loss(weights_01) + (beta/2.)*tf.nn.l2_loss(weights_12)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_hid_out = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_01) + biases_01)
  valid_prediction = tf.nn.softmax(
    tf.matmul(valid_hid_out, weights_12) +  biases_12)
  
  test_hid_out = tf.nn.relu(tf.matmul(tf_test_dataset, weights_01) + biases_01)
  test_prediction = tf.nn.softmax(
            tf.matmul(test_hid_out,weights_12) + biases_12)
        

In [30]:
overfit = True


print("Offsets: ",offset_list)
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    if not overfit:
      offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    else:
      offset = offset_list[step%len(offset_list)]
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

#Steps 10001
#test accuracy 94.1% (Neural Network) (No Overfitting)
#test accuracy 89.9% (Neural Network) (Overfitting)

Offsets:  [15958, 14751, 26542, 13569, 23686, 79248, 56272, 90653, 11217, 68634, 43230, 75549, 44776, 62585, 23691, 6094, 27525, 25837, 39670, 48951, 51865, 39897, 83480, 32883, 97918]
Initialized
Minibatch loss at step 0: 698.270691
Minibatch accuracy: 8.6%
Validation accuracy: 20.9%
Minibatch loss at step 500: 130.128571
Minibatch accuracy: 95.3%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 106.213997
Minibatch accuracy: 96.9%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 79.906113
Minibatch accuracy: 97.7%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 62.520782
Minibatch accuracy: 98.4%
Validation accuracy: 81.6%
Minibatch loss at step 2500: 48.280327
Minibatch accuracy: 97.7%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 38.645065
Minibatch accuracy: 99.2%
Validation accuracy: 81.2%
Minibatch loss at step 3500: 28.586491
Minibatch accuracy: 99.2%
Validation accuracy: 81.8%
Minibatch loss at step 4000: 22.546017
Minibatch accuracy: 99.2%
Vali

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [54]:
image_size = 28
num_labels = 10
start_lr = 0.45
num_steps = 20001
early_stopping = True
lr_decay = True
use_dropout = True
drop_rate = 0.25
beta = 1e-5
layer_sizes = [image_size*image_size,1024,512,256,num_labels]
batch_size = 128
include_l2_loss = True
activation = "relu" #relu sigmoid
loss_func = "disc" #disc or gen or disc_gen
denoising = False
lam = 0.00001
weight_init_factor = 0.01 #without this I couldn't use RELU's 
#gave NAN after 500 steps. Need small weights for multi layers

valid_accuracy_frame_size = 10

In [55]:
def get_layer_out(x,W,b,use_dropout):
  
  if use_dropout:
    if activation == "sigmoid":
      return tf.nn.dropout(tf.nn.sigmoid(tf.matmul(x,W)+b),1.-drop_rate,seed=tf.set_random_seed(1234321))
    elif activation == "relu":
      return tf.nn.dropout(tf.nn.relu(tf.matmul(x,W)+b),1.-drop_rate,seed=tf.set_random_seed(1234321))
    else:
      raise NotImplementedError
  else:
    if activation == "sigmoid":
      return tf.nn.sigmoid(tf.matmul(x,W)+b)
    elif activation == "relu":
      return tf.nn.relu(tf.matmul(x,W)+b)
    else:
      raise NotImplementedError

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
    
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))  
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables. Weights (input -> hid, hid -> output)
  weights = []
  biases = []
  biases_prime = []
  for l_i in range(len(layer_sizes)-1):
    weights.append(tf.Variable(tf.truncated_normal([layer_sizes[l_i],layer_sizes[l_i+1]])*weight_init_factor)) 
    biases.append(tf.Variable(tf.zeros([layer_sizes[l_i+1]])))    
    biases_prime.append(tf.Variable(tf.zeros([layer_sizes[l_i]])))
    
  # Training computation.
  x = tf_train_dataset
  # Use for loop to calculate the output until the output layer (output layer doesn't use dropout)
  for w,b in zip(weights[:-1],biases[:-1]):
    #print("X,W,b %s,%s,%s"%(x.eval().shape,w.eval().shape,b.eval().shape))
    x = get_layer_out(x,w,b,use_dropout)
    
  # Calculate logits for the output layer (without dropout)
  if activation == "sigmoid":
    logits = tf.matmul(x,weights[-1])+biases[-1]
  elif activation == "relu":
    logits = tf.matmul(x,weights[-1])+biases[-1]
    
  if loss_func == 'disc':
    if include_l2_loss:
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))\
            + (beta/2.)*tf.reduce_sum([tf.nn.l2_loss(w) for w in weights]) 
    else:
      loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
    
  elif loss_func == 'disc_gen':
    x_rec = x
    for w,b_prime in zip(reversed(weights[:-1]),reversed(biases_prime[:-1])):
      x_rec = get_layer_out(x_rec,tf.transpose(w),b_prime,False)
    
    if include_l2_loss:
      disc_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + (beta/2.)*tf.reduce_sum([tf.nn.l2_loss(w) for w in weights]) 
    else:
      disc_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) 
    
    gen_loss = tf.reduce_mean(tf.sub(tf_train_dataset,x_rec)**2)
    loss = disc_loss + lam*gen_loss
    
  else:
    raise NotImplementedError
  
  # Optimizer.
  global_step = tf.Variable(0)
  if lr_decay:
    learning_rate = tf.train.exponential_decay(start_lr, global_step,decay_steps=500,decay_rate=0.95)  
  else:
    learning_rate = tf.Variable(start_lr)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  inc_gstep = global_step.assign(global_step+1)

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  vx = tf_valid_dataset
  tsx = tf_test_dataset
  for w,b in zip(weights[:-1],biases[:-1]):
    vx = get_layer_out(vx,w,b,False)
    tsx = get_layer_out(tsx,w,b,False)

  if activation == 'sigmoid':
    v_logits = tf.matmul(vx, weights[-1]) + biases[-1]
    ts_logits = tf.matmul(tsx, weights[-1]) + biases[-1]
  elif activation == 'relu':
    v_logits = tf.matmul(vx, weights[-1]) + biases[-1]
    ts_logits = tf.matmul(tsx, weights[-1]) + biases[-1]
    
  valid_prediction = tf.nn.softmax(v_logits)
  test_prediction = tf.nn.softmax(ts_logits)
        
        

In [56]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Learning rate: ",start_lr)
  print("Number of Steps: ",num_steps)
  print("Early Stopping:",early_stopping)
  print("Decay learning rate: ",lr_decay)
  print("Dropout: ",use_dropout,',',drop_rate)
  print("Beta: ",include_l2_loss,',',beta)
  print("Layer sizes: ",layer_sizes)
  print("Activation: ",activation) #relu sigmoid
  print("Loss: ",loss_func) #disc or gen or disc_gen
  print("Lambda: ",lam)
  print("Weight Init Factor: ",weight_init_factor)
  print("\n")
  print("Initialized")
  
  prev_valid_accuracies = []
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, lr, l, predictions,_ = session.run(
      [optimizer, learning_rate, loss, train_prediction,inc_gstep], feed_dict=feed_dict)
        
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Learning rate: %.2f"%lr)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      v_acc = accuracy(valid_prediction.eval(), valid_labels)
      print("Validation accuracy: %.1f%%" %v_acc)
    
      if early_stopping and len(prev_valid_accuracies)>0 and v_acc < np.mean(prev_valid_accuracies[len(prev_valid_accuracies)-valid_accuracy_frame_size:])*0.9:
        print("Validation accuracy has not improved since %d steps"%valid_accuracy_frame_size*500)
        break
      prev_valid_accuracies.append(v_acc)
      
        
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))


#lr (start,decay_steps,decay_rate)
#lr (0.45,100,0.95) steps 10001 loss "disc_gen" dropout 0.5 : Takes too much time
#lr (0.45,100,0.95) steps 10001 loss "disc" dropout 0.25 : 

Learning rate:  0.45
Number of Steps:  20001
Early Stopping: True
Decay learning rate:  True
Dropout:  True , 0.25
Beta:  True , 1e-05
Layer sizes:  [784, 1024, 512, 256, 10]
Activation:  relu
Loss:  disc
Lambda:  1e-05
Weight Init Factor:  0.01


Initialized
Minibatch loss at step 0: 2.302879
Learning rate: 0.45
Minibatch accuracy: 10.2%
Validation accuracy: 8.3%
Minibatch loss at step 500: 0.425489
Learning rate: 0.43
Minibatch accuracy: 86.7%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.406803
Learning rate: 0.41
Minibatch accuracy: 88.3%
Validation accuracy: 85.6%
Minibatch loss at step 1500: 0.394135
Learning rate: 0.39
Minibatch accuracy: 89.1%
Validation accuracy: 86.7%
Minibatch loss at step 2000: 0.381361
Learning rate: 0.37
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Minibatch loss at step 2500: 0.310904
Learning rate: 0.35
Minibatch accuracy: 88.3%
Validation accuracy: 88.1%
Minibatch loss at step 3000: 0.379548
Learning rate: 0.33
Minibatch accuracy: 8

Learning rate: 0.45
Number of Steps: 10001
Early Stopping: True
Decay Learning rate: False
Dropout: True
drop_rate = 0.25
Beta: True, 1e-5
Layer sizes = [784, 1024, 512, 10]
Activation: relu
Loss: disc_gen
Lambda: 1e-05
weight Init Factor: 0.01

Test accuracy: 95.5%

==========================================================================

Learning rate:  0.45
Number of Steps:  10001
Early Stopping: True
Decay learning rate:  True
Dropout:  True , 0.25
Beta:  True , 1e-05
Layer sizes:  [784, 1024, 512, 10]
Activation:  relu
Loss:  disc
Lambda:  1e-05
Weight Init Factor:  0.01

Test accuracy: 95.5%

===========================================================================

Learning rate:  0.45
Number of Steps:  20001
Early Stopping: True
Decay learning rate:  True
Dropout:  True , 0.25
Beta:  True , 1e-05
Layer sizes:  [784, 1024, 512, 256, 10]
Activation:  relu
Loss:  disc
Lambda:  1e-05
Weight Init Factor:  0.01

Test accuracy: 96.4%

============================================================================